In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
weight1 = pd.read_csv('weight_0.0001.csv')
weight2 = pd.read_csv('weight_1e-05.csv')
weight3 = pd.read_csv('weight_1e-06.csv')
weight4 = pd.read_csv('weight_1e-07.csv')

In [5]:
weight1

,weight,feature
0,-6.314734,1
1,-0.016919,2
2,7.585033,3
3,-0.756257,4
4,4.918300,5
5,2.960866,6
6,7.681923,7
7,8.606310,8
8,2.481608,9
9,4.239114,10


In [6]:
diff_wt ={
    0.0001:weight1.weight,
    0.00001:weight2.weight,
    0.000001:weight3.weight,
    0.0000001:weight4.weight
}

In [7]:
features = pd.read_csv('features_used.csv')
x_names = np.array(features['names'])

In [8]:
x_names

array(['bedrooms', 'bathrooms', 'sqft_living', 'floors', 'waterfront',
       'view', 'grade', 'sqft_above', 'sqft_basement', 'lat',
       'sqft_living15', 'dummy'], dtype=object)

In [9]:
main_df_val = pd.read_csv('PA1_dev.csv.xls')

In [10]:
df_val = main_df_val[x_names]
df_val.head()

,bedrooms,bathrooms,sqft_living,floors,waterfront,view,grade,sqft_above,sqft_basement,lat,sqft_living15,dummy
0,3,2.25,2570,2.0,0,0,7,2170,400,47.7210,1690,1
1,3,2.00,1680,1.0,0,0,8,1680,0,47.6168,1800,1
2,3,2.50,3560,1.0,0,0,8,1860,1700,47.6007,2210,1
3,2,1.00,1160,1.0,0,0,7,860,300,47.6900,1330,1
4,3,2.00,1890,2.0,0,0,7,1890,0,47.7277,1890,1


In [11]:
df_val = df_val.drop(['dummy'],1)

In [12]:
for feature in df_val.columns:
    mn = df_val[feature].min()
    rnge = df_val[feature].max() - df_val[feature].min()
    df_val[feature] = df_val[feature].apply(lambda x : (x-mn)/rnge)

In [13]:
df_val['dummy']=main_df_val['dummy']

In [14]:
df_val.head()

,bedrooms,bathrooms,sqft_living,floors,waterfront,view,grade,sqft_above,sqft_basement,lat,sqft_living15,dummy
0,0.250,0.206897,0.165779,0.4,0.0,0.0,0.4,0.197339,0.082988,0.908027,0.184116,1
1,0.250,0.172414,0.098099,0.0,0.0,0.0,0.5,0.143016,0.000000,0.738707,0.203971,1
2,0.250,0.241379,0.241065,0.0,0.0,0.0,0.5,0.162971,0.352697,0.712545,0.277978,1
3,0.125,0.034483,0.058555,0.0,0.0,0.0,0.4,0.052106,0.062241,0.857654,0.119134,1
4,0.250,0.172414,0.114068,0.4,0.0,0.0,0.4,0.166297,0.000000,0.918915,0.220217,1


In [15]:
def calculate_sse(weight):
    print(df_val.shape,weight.shape)
    p = df_val.dot(np.array(weight)) 
    y = main_df_val['price']
    diff = p - y
    sse = (np.transpose(diff)).dot(diff)
    return sse

In [16]:
learning_rate = [0.0001,0.00001,0.000001,0.0000001]
for lr in learning_rate:
    sse = calculate_sse(diff_wt[lr])
    print("lr = ",lr," sse = ",sse)
    

(5597, 12) (12,)
lr =  0.0001  sse =  42092.23353022153
(5597, 12) (12,)
lr =  1e-05  sse =  42392.65437407539
(5597, 12) (12,)
lr =  1e-06  sse =  31865.83941045411
(5597, 12) (12,)
lr =  1e-07  sse =  26098.74267073136


In [17]:
############### thus choosing learn rate = 10**(-7)

In [18]:
main_df_train = pd.read_csv('PA1_train.csv.xls')
df_train = main_df_train.drop(['id','dummy'],1)

In [19]:
df_train.columns

Index(['date', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors',
       'waterfront', 'view', 'condition', 'grade', 'sqft_above',
       'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode', 'lat', 'long',
       'sqft_living15', 'sqft_lot15', 'price'],
      dtype='object')

In [20]:
df_train['date'] = pd.to_datetime(main_df_train['date'])
df_train['month'] = df_train['date'].apply(lambda x:x.month)
df_train['year'] =  df_train['date'].apply(lambda x:x.year)
df_train['day'] = df_train['date'].apply(lambda x:x.day)

In [21]:
df_train.columns.shape

(23,)

In [22]:
df_train = df_train.drop(['date'],1)
df_train.shape

(10000, 22)

In [23]:
df_train.columns

Index(['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors',
       'waterfront', 'view', 'condition', 'grade', 'sqft_above',
       'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode', 'lat', 'long',
       'sqft_living15', 'sqft_lot15', 'price', 'month', 'year', 'day'],
      dtype='object')

In [24]:
y_train = df_train['price']
df_train = df_train.drop(['price'],1)

In [25]:
for feature in df_train.columns:
    mn = df_train[feature].min()
    rnge = df_train[feature].max() - df_train[feature].min()
    df_train[feature] = df_train[feature].apply(lambda x : (x-mn)/rnge)

In [26]:
df_train.head()

,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,...,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,month,year,day
0,0.09375,0.275862,0.246849,0.005715,0.4,0.0,0.0,0.50,0.555556,0.276796,...,0.773913,0.0,0.368687,0.761782,0.394979,0.400000,0.011601,0.545455,0.0,0.266667
1,0.03125,0.275862,0.234244,0.002682,0.0,0.0,0.0,1.00,0.444444,0.109541,...,0.226087,0.0,0.631313,0.683127,0.112971,0.318584,0.004985,0.545455,0.0,0.566667
2,0.12500,0.172414,0.134454,0.001471,0.2,0.0,0.0,0.50,0.444444,0.150766,...,0.017391,0.0,0.722222,0.707093,0.174059,0.226549,0.003837,0.545455,0.0,0.200000
3,0.09375,0.172414,0.141807,0.005469,0.0,0.0,0.0,0.75,0.444444,0.159011,...,0.600000,0.0,0.292929,0.515522,0.300418,0.212389,0.011556,0.272727,1.0,0.900000
4,0.15625,0.241379,0.363445,0.006426,0.0,0.0,0.5,1.00,0.555556,0.243816,...,0.539130,0.0,0.035354,0.690043,0.335565,0.361062,0.011188,0.727273,0.0,0.100000


In [27]:
def check_explode(norm):
    does_explode = True
    for i in range(1,len(norm)-1):
        if norm[i]<norm[i-1]:
            does_explode = False
    return does_explode

In [28]:
df_train.describe()

,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,...,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,month,year,day
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,...,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,0.074225,0.223293,0.179645,0.008794,0.201480,0.007000,0.057350,0.602275,0.408133,0.167621,...,0.618477,0.040311,0.390369,0.649693,0.251643,0.271562,0.013884,0.508400,0.318500,0.493403
std,0.029476,0.105535,0.095728,0.024960,0.217059,0.083377,0.188983,0.163398,0.131118,0.097864,...,0.256353,0.195722,0.270295,0.223018,0.118330,0.122460,0.032441,0.282858,0.465918,0.287392
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.062500,0.172414,0.111345,0.002704,0.000000,0.000000,0.000000,0.500000,0.333333,0.096584,...,0.443478,0.000000,0.161616,0.505549,0.155649,0.182301,0.005091,0.272727,0.000000,0.233333
50%,0.062500,0.241379,0.162815,0.004269,0.200000,0.000000,0.000000,0.500000,0.333333,0.140165,...,0.660870,0.000000,0.323232,0.668168,0.237657,0.246018,0.007995,0.454545,0.000000,0.500000
75%,0.093750,0.275862,0.228992,0.006166,0.400000,0.000000,0.000000,0.750000,0.444444,0.219081,...,0.843478,0.000000,0.587121,0.838829,0.325523,0.339823,0.010839,0.727273,1.000000,0.733333
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [29]:
all_info={}

In [30]:
import math
def calculate_weights_and_sse(df,max_iteration,check_iter_for_explode,lmbda,learning_rate=0.0000001):
    
    weight = np.array(np.random.randn(1,21)[0])
    
    number_of_iteration = 0
    sse_for_each_iter = [] 
    norm_for_this_lr = []
    while True:
        diff_in_y = df.dot(weight)-y_train
        
        
        gradient = (np.transpose(df)).dot(diff_in_y) + 2*lmbda*weight
        
        weight = weight - learning_rate*gradient 
        norm_for_gradient = math.sqrt(gradient.dot(gradient))
        
        sse = (np.transpose(diff_in_y)).dot(diff_in_y) + lmbda*(norm_for_gradient**2)
        
        
        
        
        
        #print('number of iteration',number_of_iteration,norm_for_gradient,'   sse =',sse)
        
        
        sse_for_each_iter.append(sse)
        
        norm_for_this_lr.append(norm_for_gradient)
        
        
        number_of_iteration = number_of_iteration + 1
        
        
        if(norm_for_gradient<=0.5):
            
            all_info[learning_rate] = {"whether converged":True,
                               "weights":weight,
                               "sse_for_each_iteration":sse_for_each_iter,
                               "number_of_iteration_used":number_of_iteration
                              }
            print("########################")
            print("number of itertion used for lmbda =",lmbda , " is ",number_of_iteration , 'and sse is ' ,sse)
            print("########################")
            break
            
            
        if(number_of_iteration == check_iter_for_explode and check_explode(norm_for_this_lr)):
            
            print('Gradient explodes for learning_rate =',learning_rate)
            all_info[learning_rate] = {"whether converged":False,
                               "weights":np.array(np.zeros(12)),
                               "sse_for_each_iteration":np.array([-1]*12),
                               "number_of_iteration_used":number_of_iteration
                              }
            break
            
        if(number_of_iteration==max_iteration):
    
            all_info[learning_rate] = {"whether converged":True,
                               "weights":weight,
                               "sse_for_each_iteration":sse_for_each_iter,
                               "number_of_iteration_used":number_of_iteration
                              }
            print("converges but takes too many iterations  for lambda =",lmbda)
            break
            
            
        if (number_of_iteration%5000)==0:
            print('number of iteration',number_of_iteration,norm_for_gradient,sse)
    
    print('checked for lambda = ',lmbda," now returning")
    return
        
            


In [31]:
lmbda_values = [0.001,0.01,0.1,0,1,10,100]
for lmb in lmbda_values[:1]:
    print('calculating everything for lmbda = ',lmb,'\n\n')
    calculate_weights_and_sse(df_train,200000,100,lmb)

calculating everything for lmbda =  0.001 


number of iteration 5000 4929.086984412564 118475.22079160326
number of iteration 10000 3745.964951216033 89582.8975253561
number of iteration 15000 2906.6894734379994 73029.79543038517
number of iteration 20000 2299.632671254664 63144.84033301205
number of iteration 25000 1854.4429968144852 57011.480624601485
number of iteration 30000 1524.437165404074 53060.343814746135
number of iteration 35000 1277.5337260368933 50417.042025941235
number of iteration 40000 1091.083967471048 48580.01132298735
number of iteration 45000 948.7895550730377 47254.13564525898
number of iteration 50000 838.7946877734752 46261.703873772836
number of iteration 55000 752.4489708988798 45493.32054157835
number of iteration 60000 683.4576930868585 44880.15700512493
number of iteration 65000 627.2655798519237 44377.926429095234
number of iteration 70000 580.5944859845746 43957.45535032502
number of iteration 75000 541.0936243691934 43599.04347044154
number of iteratio

In [32]:
lmbda_values = [0.001,0.01,0.1,0,1,10,100]
for lmb in lmbda_values[1:3]:
    print('calculating everything for lmbda = ',lmb,'\n\n')
    calculate_weights_and_sse(df_train,200000,100,lmb)

calculating everything for lmbda =  0.01 


number of iteration 5000 4375.469839911667 275601.6661877469
number of iteration 10000 3392.9824247409774 184306.9640514679
number of iteration 15000 2667.681882538247 131229.77190748096
number of iteration 20000 2125.8108853382496 99547.88076547685
number of iteration 25000 1717.7963980864706 80194.37285872233
number of iteration 30000 1408.6211083378191 68098.41704053682
number of iteration 35000 1172.931850449745 60356.55748897807
number of iteration 40000 992.0705915887722 55274.66010049434
number of iteration 45000 852.1676070869582 51847.81240499295
number of iteration 50000 742.8696660177064 49470.782121676544
number of iteration 55000 656.455824319321 47773.584737375946
number of iteration 60000 587.1939014271605 46526.56030860162
number of iteration 65000 530.8532425613543 45584.81337022188
number of iteration 70000 484.32756483393143 44855.29704234263
number of iteration 75000 445.3423977869173 44277.09578506747
number of iteration 

In [33]:
lmbda_values = [0.001,0.01,0.1,1,10,100]
for lmb in lmbda_values[3:5]:
    print('calculating everything for lmbda = ',lmb,'\n\n')
    calculate_weights_and_sse(df_train,200000,100,lmb)

calculating everything for lmbda =  1 


number of iteration 5000 4353.212457460204 19038776.18768693
number of iteration 10000 3416.835703302781 11748113.598433122
number of iteration 15000 2734.2641271670473 7540127.953982822
number of iteration 20000 2219.5222172772133 4984083.398438944
number of iteration 25000 1827.3295585461096 3392842.6694733836
number of iteration 30000 1526.5046765621223 2381106.633907882
number of iteration 35000 1294.1991329164546 1723842.3213207782
number of iteration 40000 1113.3465351950979 1286971.1915861503
number of iteration 45000 971.1275458101607 989421.8388470893
number of iteration 50000 857.92809773394 781526.4573807655
number of iteration 55000 766.5774723170601 632456.7690226875
number of iteration 60000 691.7631457184369 522810.65681435645
number of iteration 65000 629.5715565965913 440189.3078358174
number of iteration 70000 577.1282480462506 376533.6386350577
number of iteration 75000 532.320306442518 326506.10808250646
number of iteration 8

KeyboardInterrupt: 

In [ ]:
calculate_weights_and_sse(df_train,200000,100,100)

In [ ]:
###Training SSE
for l in lmbda_values:
    data_l = all_info[l]
    sse_value_train=data_l['sse_for_each_iteration']
    length = len(sse_value_train)
    print("lambda = ",l," and sse = ",sse_value_train[length-1] ," for 200000 iterations")